In [183]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
import warnings

In [25]:
!pip install catboost xgboost


  Using cached catboost-1.2.7-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
Using cached catboost-1.2.7-cp312-cp312-win_amd64.whl (101.7 MB)
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB 330.3 kB/s eta 0:07:34
   ---------------------------------------- 0.1/150.0 MB 731.4 kB/s eta 0:03:25
   ---------------------------------------- 0.2/150.0 MB 985.7 kB/s eta 0:02:32
   ---------------------------------------- 0.3/150.0 MB 1.2 MB/s eta 0:02:03
   ---------------------------------------- 0.4/150.0 MB 1.4 MB/s eta 0:01:48
   ---------------------------------------- 0.4/150.0 MB 1.4 MB/s eta 0:01:49
   ---------------------------------------- 0.7/150.0 MB 2.1 MB/s eta 0:01:13
   ----------------------------

In [185]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [251]:
df = pd.read_csv('cleaned_cars_dekho.csv')

In [253]:
df['super_charger'] = df['super_charger'].astype(int)

In [255]:
df['oem'] = df['oem'].replace('mahindra ssangyong', 'mahindra')
df['oem'] = df['oem'].replace('mahindra renault', 'mahindra')

In [257]:
# To remove extrem outliers
Q1 = df['selling_price'].quantile(0.25)
Q3 = df['selling_price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df['selling_price'] >= lower_bound) & (df['selling_price'] <= upper_bound)]

In [259]:
df.shape

(34564, 41)

In [261]:

X = df[["myear",
"oem",
"model",
"gear_box",
"seats",
"steering_type",
"no_of_cylinder",
"super_charger",
"top_speed",
"max_power_delivered",
"max_torque_delivered",
"feature_score"]]

In [263]:
X.head()
X.shape

(34564, 12)

In [265]:
y = df[['selling_price']]
y.head()
y.shape

(34564, 1)

In [267]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include=['object']).columns
print(num_features,cat_features)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

Index(['myear', 'seats', 'no_of_cylinder', 'super_charger', 'top_speed',
       'max_power_delivered', 'max_torque_delivered', 'feature_score'],
      dtype='object') Index(['oem', 'model', 'gear_box', 'steering_type'], dtype='object')


In [269]:
newTrainDf =  pd.concat([X, y], axis=1)
newTrainDf.to_csv('training_encoded.csv', index=False)

In [271]:
X = preprocessor.fit_transform(X)


In [273]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)
X_train.shape, X_test.shape

((24194, 352), (10370, 352))

In [275]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [277]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 126905.5721
- Mean Absolute Error: 92992.5121
- R2 Score: 0.8696
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 130371.3473
- Mean Absolute Error: 94609.6274
- R2 Score: 0.8637




C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 79343261123770.4, tolerance: 298813698344.0077
  model = cd_fast.sparse_enet_coordinate_descent(


Lasso
Model performance for Training set
- Root Mean Squared Error: 126912.4565
- Mean Absolute Error: 93023.4622
- R2 Score: 0.8696
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 130428.8672
- Mean Absolute Error: 94578.0218
- R2 Score: 0.8636


Ridge
Model performance for Training set
- Root Mean Squared Error: 127287.6328
- Mean Absolute Error: 93311.6313
- R2 Score: 0.8688
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 130199.7272
- Mean Absolute Error: 94487.8685
- R2 Score: 0.8640


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 99819.8357
- Mean Absolute Error: 69806.2570
- R2 Score: 0.9193
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 110612.5181
- Mean Absolute Error: 77584.4410
- R2 Score: 0.9019


Decision Tree
Model performance for Training set
- Root Mean Squared Error: 83820.6710
- Mean Absolute Err

C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Regressor
Model performance for Training set
- Root Mean Squared Error: 85753.0871
- Mean Absolute Error: 59914.3399
- R2 Score: 0.9405
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 105882.8095
- Mean Absolute Error: 74309.3686
- R2 Score: 0.9101


XGBRegressor
Model performance for Training set
- Root Mean Squared Error: 96994.9663
- Mean Absolute Error: 69951.5990
- R2 Score: 0.9238
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 104594.1460
- Mean Absolute Error: 74482.6120
- R2 Score: 0.9123


CatBoosting Regressor
Model performance for Training set
- Root Mean Squared Error: 98152.0794
- Mean Absolute Error: 71065.7932
- R2 Score: 0.9220
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 103531.6285
- Mean Absolute Error: 73965.3720
- R2 Score: 0.9140




C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoost Regressor
Model performance for Training set
- Root Mean Squared Error: 180378.6075
- Mean Absolute Error: 143713.2371
- R2 Score: 0.7366
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 181416.2995
- Mean Absolute Error: 144840.0585
- R2 Score: 0.7361




In [279]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
7,CatBoosting Regressor,0.914038
6,XGBRegressor,0.912265
5,Random Forest Regressor,0.910089
3,K-Neighbors Regressor,0.901877
4,Decision Tree,0.897810
2,Ridge,0.864050
0,Linear Regression,0.863691
1,Lasso,0.863571
8,AdaBoost Regressor,0.736055


ValueError: could not convert string to float: 'maruti'